In [1]:
import pandas as pd
import numpy as np
import re
import math
import pickle
import glob
import xgboost
import tsfresh
import os
import hockey_scraper
import glob
from hmmlearn import hmm
from datetime import datetime
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from pprint import pprint
from scipy import stats
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score, log_loss
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_context('notebook')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 100, 'linewidths':0}
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.mode.chained_assignment = None  # default='warn'
np.random.seed(42)

# Variables
scrape = False

### Example

In [2]:
model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.6, 0.3, 0.1])
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))
X, Z = model.sample(100)

### Real Data

In [3]:
all_stats = {}

In [4]:
filenames = glob.glob(os.path.join('../LSTM-Neural-Network-for-Time-Series-Prediction/data', '*.csv'))
for f in filenames:
    print(f.split('/')[-1])
    if not os.path.basename(f) in all_stats.keys():
        data = pd.read_csv(f)
        data['Date'] = pd.to_datetime(data['Date'])
        data = data[data['Date'] < pd.datetime(2019, 10, 1)]
        data = data[data['Date'] > pd.datetime(2016, 10, 1)]
        if data.shape[0] < 25:
            all_stats[f.split('/')[-1]] = 0
            continue
        X = np.array(data['Total_Points']).reshape(-1, 1)
        # X = np.array(pd.read_csv('../LSTM-Neural-Network-for-Time-Series-Prediction/data/NATHAN-MACKINNON-8477492.csv')['Total_Points']).reshape(-1, 1)
        if np.unique(X).shape == 1:
            all_stats[f.split('/')[-1]] = 0
            continue
        if X.shape[0] == 0:
            all_stats[f.split('/')[-1]] = 0
            continue
        n_states = int(X.max() + 1)
        model = hmm.GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=1000)
        model = model.fit(X)
        model.monitor_.converged

        points = np.array([])
        for i in np.arange(0,500):
            tmp = model.sample(82)[0].sum()
            points = np.append(points, tmp)

        statistics = stats.describe(points)
        all_stats[f.split('/')[-1]] = statistics

RYAN-KESLER-8470616.csv


True

ETHAN-BEAR-8478451.csv
LOGAN-SHAW-8476400.csv


True

NELSON-NOGIER-8478031.csv
MIKE-SMITH-8469608.csv


...

True

BRANDON-PIRRI-8475204.csv


True

MIKE-MATHESON-8476875.csv


True

In [5]:
all_stats_df = pd.DataFrame.from_dict(all_stats, orient='columns').T
all_stats_df.columns = ['nobs', 'minmax', 'mean', 'variance', 'skewness', 'kurtosis']
all_stats_df.to_pickle('predictions_hmm_3years_for_1920.pkl')

In [6]:
all_stats_df

,nobs,minmax,mean,variance,skewness,kurtosis
RYAN-KESLER-8470616.csv,500,"(19.0486693136375, 57.939195964332185)",35.7392,34.5478,0.135661,0.119872
ETHAN-BEAR-8478451.csv,0,0,0,0,0,0
LOGAN-SHAW-8476400.csv,500,"(3.9503219831557255, 32.00679263430701)",15.5919,19.4693,0.243871,0.142775
NELSON-NOGIER-8478031.csv,0,0,0,0,0,0
MIKE-SMITH-8469608.csv,500,"(-0.1488302137364587, 9.169448730565826)",2.69421,2.3494,0.469817,0.222445
MACKENZIE-BLACKWOOD-8478406.csv,0,0,0,0,0,0
GABRIEL-BOURQUE-8475268.csv,500,"(3.90282959342707, 26.06032901641795)",13.1227,13.8762,0.228797,0.139418
FILIP-ZADINA-8480821.csv,0,0,0,0,0,0
CONNOR-BROWN-8477015.csv,500,"(16.0092994907697, 48.94189239783822)",30.6007,32.4121,0.173803,0.109716
PIERRE-EDOUARD-BELLEMARE-8477930.csv,500,"(4.9168322240219995, 28.17549279573322)",14.0618,12.7755,0.413403,0.321952


In [ ]:
data = pd.read_csv('../LSTM-Neural-Network-for-Time-Series-Prediction/data/NATHAN-MACKINNON-8477492.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data[data['Date'] < pd.datetime(2018, 10, 1)]
X = np.array(data['Total_Points']).reshape(-1, 1)
# X = np.array(pd.read_csv('../LSTM-Neural-Network-for-Time-Series-Prediction/data/NATHAN-MACKINNON-8477492.csv')['Total_Points']).reshape(-1, 1)
n_states = int(X.max() + 1)
model = hmm.GaussianHMM(n_components=n_states, covariance_type="full", n_iter=1000)
model = model.fit(X)
model.monitor_.converged

In [ ]:
points = np.array([])
for i in np.arange(0,500):
    tmp = model.sample(82)[0].sum()
    points = np.append(points, tmp)

In [ ]:
stats.describe(points)